# Analyse de la qualité des données

## Prise de connaissance avec le jeu de données

### Chargement des données brut

In [ ]:
from getting_started import df_patient, df_pcr, pd

Conversion de chaque attribut du référentiel en un type de données plus spécifique.

In [ ]:
df_patient = df_patient.convert_dtypes()

df_patient.info()

Conversion de chaque attribut de l'échantillon en un type de données plus spécifique.

In [ ]:
df_pcr = df_pcr.convert_dtypes()

df_pcr.info()

### Présence de doublons sur l'identifiant

In [ ]:
with_duplicated_id = df_patient.patient_id.duplicated(keep=False)

df_patient[with_duplicated_id].sort_values("patient_id")

Il y a 403 patients qui doublonnent sur l'identifiant.

### Présence de doublons sur l'ensemble des attributs

In [ ]:
with_same_attributes = df_patient.drop(columns="patient_id").duplicated(keep=False)

df_patient[with_same_attributes].sort_values(by=["surname", "given_name"])

Il y 22 patients qui doublonnent sur tous les attributs hors identifiant.

### Nombre d'attributs non renseignés par patient

In [ ]:
df_patient.isna().sum(axis="columns").value_counts().sort_index().plot.bar(xlabel="attributs non renseignés", ylabel="patients")

La majorité des patients ont jusqu'à 2 attributs non renseignés.

Il y aura des cas extrêmes à rapprocher avec 5 voire 6 attributs manquants.

### Répartition des valeurs du test PCR

In [ ]:
df_pcr.pcr.value_counts()

Deux conventions sont utilisées pour représenter les deux valeurs possibles d'un test PCR (négatif ou positif) : `N / P` et `Negative / Positive`.

Il faudra normaliser ces résultats dans une variable catégorielle ordonnée.

### Exhaustivité du référentiel

In [ ]:
df_pcr.patient_id.isin(df_patient.patient_id).all()

L'intégralité des identifiants associés aux tests de l'échantillon sont présents dans le référenciel.

## Analyse du référenciel de patients

### Nom et prénom

#### Valeurs manquantes

In [ ]:
df_na_in_patient_name = df_patient[["surname", "given_name"]].isna()

df_na_in_patient_name.value_counts()

Il y a 861 patients dont le nom et / ou prénom ne sont pas renseignés.

#### Fautes typographiques

In [ ]:
from jellyfish import damerau_levenshtein_distance

df_patient["full_name"] = df_patient.agg(
    lambda x: f"{x.given_name} {x.surname}", axis="columns")

df_full_name = df_patient[["patient_id", "full_name", "phone_number"]].dropna()
df_full_name = df_full_name.merge(df_full_name, on="phone_number")
df_full_name = df_full_name[df_full_name.patient_id_x != df_full_name.patient_id_y]
df_full_name["linked_ids"] = df_full_name[["patient_id_x", "patient_id_y"]].apply(
    lambda row: tuple(sorted(row)), axis="columns")
df_full_name.drop_duplicates("linked_ids", inplace=True)
df_full_name["distance"] = df_full_name.apply(
    lambda row: damerau_levenshtein_distance(row.full_name_x, row.full_name_y), axis="columns")

df_full_name = df_full_name[df_full_name.distance >= 1]

Cette cellule procède à un rapprochement des patients partageant un même numéro de téléphone et calcule la similarité entre leurs noms complets sous forme d'une distance.

Le numéro de téléphone, quand il est fourni, est un attribut à forte valeur distinctive d'où son utilisation dans le cross-join.

La distance choisie est Damerau-Levenshtein car elle est particulièrement adaptée pour détecter les fautes humaines qui peuvent arriver lors de la saisie répétée de texte brut.

In [ ]:
df_full_name[df_full_name.distance <= 2].sort_values("phone_number").head(20)

Dans le cas d'une distance faible, on retrouve les fautes typographiques suivantes :

- Ajout de caractère (schumajnn -> schumann)
- Omission de caractère (jaob -> jacob)
- Substitution de caractère (martincvic -> martinovic)
- Transposition de caractère (taaila -> taalia)

### Autres différences

In [ ]:
df_full_name[df_full_name.distance > 2].sort_values("phone_number").head(20)

Pour les cas où la distance est plus forte, on relève d'autres différences :

- Faute lexicale (wight -> white)
- Diminutif du prénom (thomas -> to, anastasia -> stacia)
- Substitution du prénom (james -> jim, emiily -> millie)
- Inversion des nom et prénom
- Omission du prénom ou du nom

## Âge et date de naissance

### Valeurs manquantes

In [ ]:
df_patient[["date_of_birth", "age"]].isna().value_counts()

### Répartition des âges

In [ ]:
df_patient.age.describe()

In [ ]:
df_patient_age = pd.cut(df_patient.age.dropna(), bins=range(0, 101, 10), right=False)

df_patient_age.value_counts().sort_index().plot.bar(xlabel="catégorie d'âge", ylabel="patients")

In [ ]:
df_patient[df_patient.age < 10].age.value_counts().sort_index().plot.bar(xlabel="âge", ylabel="patients")

On remarquera la présence de quelques patients en très bas âge (inférieur à 6 ans).

Je n'ai pas connaissance d'un quelconque âge minimum pour effectuer un test PCR (à confirmer auprès d'un expert).

### Dates de naissance invalides

La date de naissance est stockée au format `YYYYMMDD` dans une valeur entière.

On la convertit en `datetime` afin de la valider.

In [ ]:
dob_datetime = pd.to_datetime(df_patient.date_of_birth, format="%Y%M%d", errors="coerce")


Pour celles qui ne passent pas la conversion, on procède manuellement à l'extraction de l'année, du mois et du jour.

In [ ]:
dob_invalid = (
    df_patient[dob_datetime.isna()].date_of_birth.dropna().astype(str)
        .str.extract(r"(\d{4})(\d{2})(\d{2})", expand=True)
        .rename(columns={0: "year", 1: "month", 2: "day"})
)

dob_invalid.sample(10, random_state=42)

L'échantillon montre la présence de valeurs aberrantes pour le mois et / ou le jour.

Il y a 106 patients pour lesquels la date de naissance est non conforme.

In [ ]:
len(dob_invalid)

### Incohérence entre date de naissance et âge renseigné

Pour les patients dont la date de naissance est valide, on calcule l'âge en fin d'année 2020.

In [ ]:
age_from_dob = (pd.Timestamp("2020-12-31") - dob_datetime).dt.days.floordiv(365.25).rename("age_from_dob")

dob_and_ages = (
    df_patient[["date_of_birth", "age"]]
        .merge(age_from_dob, left_index=True, right_index=True)
        .dropna().convert_dtypes()
)

dob_and_ages.sample(10, random_state=42)

L'échantillon montre une absence de cohérence entre l'âge renseigné et l'âge calculé à partir de la date de naissance.

Même avec une tolérance d'une année, seuls 342 d'entre eux ont un âge cohérent.

In [ ]:
from numpy import isclose

isclose(dob_and_ages.age.astype(int), dob_and_ages.age_from_dob.astype(int), atol=1).sum()

## Numéro de téléphone

En [Australie](https://info.australia.gov.au/about-australia/facts-and-figures/telephone-country-and-area-codes), les numéro de téléphone sont composés d'un indicatif de zone géographique sur 2 chiffres, suivi d'un indicatif local sur 8 chiffres.

Extraction des indicatifs de zone et locaux.

In [ ]:
df_patient_phone_number = df_patient.phone_number.dropna().str.extract(r"(\d{2})\s(\d{8})", expand=True).rename(columns={0: "area", 1: "local"})

df_patient_phone_number.sample(10, random_state=42)

Tous les numéro de téléphone renseignés ont un indicatif de zone correct.

In [ ]:
df_patient_phone_number.area.isin(["02", "03", "04", "07", "08"]).all()

## Adresse personnelle

### Numéro de rue

La numérotation des rues démarrent à 1. Le minimum de 0 est une valeur erronée.

In [ ]:
df_patient.street_number.dropna().astype(int).describe()

Analyse du nombre de chiffres composant le numéro de rue.

In [ ]:
street_number = df_patient.street_number.fillna(0).dropna().astype(int)
street_number["number_of_digits"] = street_number.astype(str).str.len()

street_number.number_of_digits.value_counts().sort_index().plot.bar(xlabel="digits in street number", ylabel="patients")


Il y a quelques patients avec un numéro de rue exceptionnellement élevé (nombre de chiffres >= 5).

On analyse les patients correspondant.

In [ ]:
df_patient_with_large_street_number = df_patient.loc[street_number.number_of_digits >= 5]

df_patient_with_large_street_number[["patient_id", "street_number", "address_1", "phone_number"]]

Les patients ont tous un numéro de téléphone renseigné.

On peut donc effectuer un rapprochement par celui-ci afin d'identifier une possible erreur de saisie.

In [ ]:
df = df_patient_with_large_street_number[
    ["patient_id", "street_number", "address_1", "phone_number"]
].merge(
    df_patient[["patient_id", "street_number", "address_1", "phone_number"]],
    on="phone_number"
)
df.loc[df.patient_id_x != df.patient_id_y]

Le numéro de rue à 6 chiffres est une erreur de saisie (342951 -> 3429).

Idem pour l'un des numéro de rue à 5 chiffres (10030 -> 1000).

### Adresse

#### Taux de remplissage

In [ ]:
df_patient[["address_1", "address_2"]].notna().value_counts(normalize=True).sort_index()

- 96% des adresses ont au moins le premier champ d'adresse renseigné
- 38% ont les deux champs renseignés

#### Fautes de saisie

Rapprochement des adresses de patient partageant un même numéro de téléphone.

In [ ]:
address = (
    df_patient[["patient_id", "address_1", "address_2", "phone_number"]]
        .dropna(subset=["phone_number"])
        .fillna("<NA>")
)
address["full_address"] = address[["address_1", "address_2"]].apply(", ".join, axis=1)
address = address.merge(address, on="phone_number")
address = address.loc[
    (address.patient_id_x != address.patient_id_y) & \
    (address.full_address_x != address.full_address_y)
]

address.sort_values(by=["phone_number", "patient_id_x"]).head(10)

Calcul de la distance d'édition.

In [ ]:
address["distance"] = address.apply(
    lambda x: damerau_levenshtein_distance(x.full_address_x, x.full_address_y),
    axis=1
)

address.distance.value_counts().sort_index().plot.bar()

Fautes typographiques.

In [ ]:
address.loc[address.distance <= 2, ["full_address_x", "full_address_y", "distance"]].sample(20, random_state=42)

In [ ]:
address.loc[address.distance == 3, ["full_address_x", "full_address_y", "distance"]]

In [ ]:
address.loc[address.distance == 4, ["full_address_x", "full_address_y", "distance"]]

Autres fautes.


In [ ]:
address.loc[address.distance > 4, ["full_address_x", "full_address_y", "distance"]].sample(30, random_state=42)

Omission du premier champ d'adresse.

In [ ]:
address.loc[8818, ["full_address_x", "full_address_y"]].to_list()

Omission du second champ d'adresse.

In [ ]:
address.loc[940, ["full_address_x", "full_address_y"]].to_list()

Second champ d'adresse différent.

In [ ]:
address.loc[1572, ["full_address_x", "full_address_y"]].to_list()

Adresse complète différente.

In [ ]:
address.loc[3535, ["full_address_x", "full_address_y"]].to_list()

### Quartier

#### Taux de remplissage dans l'adresse

In [ ]:
df_patient[["suburb", "address_1", "address_2"]].notna().value_counts(normalize=True).sort_index()

Le quartier est le plus souvent renseigné en complément du premier champ d'adresse.

Il y a quelques rares cas où il est fourni seul, ou avec le second champ d'adresse seulement.

#### Fautes de saisie

Certains couples quartier / postcode ont été inversées lors de la saisie.

In [ ]:
df_patient.loc[df_patient.suburb.str.contains(r"\d")]

Pour ces patients, il faut procéder à l'inversion des valeurs du quartier et du code postal.

Le code postal peut contenir un caractère alphabétique qu'il faudra supprimer (exemple `467l0` -> `4670`).

In [ ]:
swapped_suburb_postcode = df_patient.loc[df_patient.suburb.str.contains(r"\d"), ["suburb", "postcode"]]

swapped_suburb_postcode["suburb"], swapped_suburb_postcode["postcode"] = \
    swapped_suburb_postcode["postcode"], swapped_suburb_postcode["suburb"].str.replace(r"[a-z]", "")

df_patient.update(swapped_suburb_postcode)

df_patient.iloc[swapped_suburb_postcode.index]

### Code postal

#### Taux de remplissage dans l'adresse

In [ ]:
df_patient[["postcode", "suburb", "state"]].notna().value_counts(sort=False)

Le code postal est très souvent renseigné, accompagné du quartier et / ou de l'état.   

#### Validation

In [ ]:
df_patient.postcode.dropna().str.len().value_counts()

Chargement du référenciel de codes postaux valides par état.

In [ ]:
df_state_postcode = pd.read_csv("state_postcode.csv").convert_dtypes()

df_state_postcode

Validation des valeurs de codes postaux.

In [ ]:
valid_postcode = pd.arrays.IntervalArray.from_arrays(
    left=df_state_postcode.postcode_min_range.astype(int),
    right=df_state_postcode.postcode_max_range.astype(int),
    closed="both",
)

df_postcode = df_patient[["patient_id", "postcode"]].dropna(subset=["postcode"])

df_postcode["is_valid"] = df_postcode.postcode.apply(
    lambda x: valid_postcode.contains(int(x)).any()
)

df_postcode[~df_postcode.is_valid]

Cohérence avec l'état renseigné.

In [ ]:
valid_postcode_per_state = {
    state: pd.arrays.IntervalArray.from_arrays(
        left=postcode.postcode_min_range,
        right=postcode.postcode_max_range,
        closed="both",
    )
    for state, postcode in df_state_postcode.groupby(by="state").agg(tuple).iterrows()
}

df_patient_state_postcode = (
    df_patient[["patient_id", "state", "postcode"]]
        .dropna(subset=["state", "postcode"])
)

df_patient_state_postcode["is_coherent_with_postcode"] = df_patient_state_postcode.apply(
    lambda x: valid_postcode_per_state[x.state].contains(int(x.postcode)).any(),
    axis=1,
)

### État

Répartition des valeurs

In [ ]:
state_counts = df_patient.state.value_counts()

On retrouve les codes des 8 états de l'Australie.

In [ ]:
state_counts.head(8)

Et 94 autres codes d'état à rectifier.

In [ ]:
state_counts[8:]

On effectue un premier rapprochement par une distance d'édition très courte.

In [ ]:
from itertools import product
from jellyfish import damerau_levenshtein_distance

states = state_counts[:8].index.tolist()
codes = state_counts[8:].index.tolist()

code_to_state = pd.DataFrame(
    data=[damerau_levenshtein_distance(c, s) for c, s in product(codes, states)],
    index=pd.MultiIndex.from_product([codes, states], names=["code", "state"]),
    columns=["distance"],
)

code_to_state = (
    code_to_state[code_to_state.distance == 1]
        .drop(columns="distance")
        .reset_index("code")
        .drop_duplicates("code", keep=False)
        .reset_index()
        .set_index("code")
)["state"]

code_to_state.to_dict()

Correction des états.

In [ ]:
df_patient.state.replace(code_to_state.to_dict(), inplace=True)

Liste des codes restant à rapprocher.

In [ ]:
state_counts = df_patient.state.value_counts()

state_counts[8:]

On constate qu'ils ont tous un code postal renseigné.

In [ ]:
invalid_states = state_counts[8:].index.to_list()

df_patient_with_invalid_state = df_patient.loc[df_patient.state.isin(invalid_states)]

df_patient_with_invalid_state[["state", "postcode"]].count()

Utilisation d'un service de géocodage pour déduire l'état à partir du code postal.

In [ ]:
from geopy.geocoders import Nominatim

geocoder = Nominatim(user_agent="inria-aphp-assignment")
df_patient_with_invalid_state["nominatim"] = df_patient_with_invalid_state.apply(
    lambda row: geocoder.geocode(f"{row.postcode}, australia").address, axis=1)

df_patient_with_invalid_state[["postcode", "state", "nominatim"]].head()

On utilise `geopy` qui fournit une interface commune vers plusieurs services de géocodage, dont Nominatim (OpenStreetMap) qui est gratuit. Des limites de débits d'appliquent et la réponse de l'API est lente, donc il ne faut pas trop en abuser.

On extrait le libellé complet de l'état de l'adresse géocodée par Nominatim et le recode.

In [ ]:
df_patient_with_invalid_state["state_from_nominatim"] = (
    df_patient_with_invalid_state.nominatim
        .str.extract(r"(\w[\w\s]*),\s\d{4}", expand=False)
        .str.lower()
        .replace({
            "australian capital territory": "act",
            "queensland": "qld",
            "new south wales": "nsw",
            "northern territory": "nt",
            "south australia": "sa",
            "victoria": "vic",
            "tasmania": "ta",
            "western australia": "wa",
        })
)

df_patient_with_invalid_state[["postcode", "state", "state_from_nominatim"]]

Correction des états

In [ ]:
df_patient.state.iloc[df_patient_with_invalid_state.index] = df_patient_with_invalid_state.state_from_nominatim

Validation des états

In [ ]:
df_patient.state.value_counts()

Il n'y a plus de codes d'état incorrects.

## Sanitize postcode and state

Assume postcode is more reliable than state.
Test all postcodes are valid.
Case postcode invalid, try swap with suburb.
Test some state are invalid.
Normalize state with typos.
For missing or invalid states, guess from postcode.
Keep state if postcode invalid.


In [ ]:
df_patient.state = df_patient.state.str.upper()
df_patient_invalid_postcode = df_patient[df_patient.state.isin(["NSW", "VIC", "QLD", "WA", "SA", "TAS", "ACT", "NT"])]
df_patient_vic = df_patient_invalid_postcode.loc[df_patient_invalid_postcode.state == "VIC"]
df_patient_vic.loc[~df_patient_vic.postcode.str.match(r"[3|8]\d{3}")].head()